# Incremental Watermark merge

Merges watermark values from incremental load parquet files back into the central watermarks configuration file.

**Process:**
1. Receives parameters from pipeline (source, run_id, paths)
2. Reads watermark updates from runtime parquet files
3. Updates central watermarks.json with new values

**Parameters:**
- `source`: Source system name
- `run_id`: Current run identifier
- `wm_configpath`: Path to central watermarks config
- `wm_folder`: Runtime folder containing watermark parquet files

In [ ]:
import os
import json

# Detect mode and set base path
base_path = '/lakehouse/default/Files' if os.path.exists('/lakehouse/default') else 'Files'

wm_configpath_full = f'{base_path}/{wm_configpath}'
wm_folder_full = f'{base_path}/{wm_folder}'

print(f"Source: {source} | Run: {run_id}")
print(f"Watermark folder: {wm_folder_full}")

## Load Watermarks Configuration

Reads the central watermarks JSON file and locates the configuration for the current source system.

In [ ]:
# Load watermarks config
with open(wm_configpath_full, 'r') as f:
    cfg = json.load(f)

# Find source in config
sources = cfg.get("source", [])
src = next((s for s in sources if s.get("name") == source), None)

if not src:
    raise ValueError(f"Source '{source}' not found in config")

tables_map = src.get("tables", {})
print(f"Current watermarks: {len(tables_map)} tables")

## Read Watermark Updates with Spark

Uses Spark to read all watermark parquet files in parallel and extracts the maximum (most recent) watermark value per table.

**Optimization:** Replaces sequential file reading with distributed Spark processing for 10-100x performance improvement.

In [ ]:
# Read all parquet files with PyArrow (workaround for OneLake issue)
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import os

if not os.path.exists(wm_folder_full):
    print("No watermark folder - skipping")
    updates = {}
else:
    updates = {}
    
    # Loop through table folders
    for table_name in os.listdir(wm_folder_full):
        table_path = os.path.join(wm_folder_full, table_name)
        
        if os.path.isdir(table_path):
            try:
                # Read with PyArrow
                parquet_files = [f for f in os.listdir(table_path) if f.endswith('.parquet')]
                
                if parquet_files:
                    # Read all parquets in this table folder
                    dataset = ds.dataset(table_path, format='parquet')
                    table = dataset.to_table()
                    
                    # Get max watermark if data exists
                    if table.num_rows > 0 and 'watermark' in table.column_names:
                        wm_column = table.column('watermark')
                        
                        # Find max non-null watermark
                        valid_watermarks = [wm.as_py() for wm in wm_column if wm.as_py() is not None and wm.as_py() != '']
                        
                        if valid_watermarks:
                            max_wm = max(valid_watermarks)
                            updates[table_name] = max_wm
                            print(f"  {table_name}: {max_wm}")
            
            except Exception as e:
                print(f"  ⚠️  Skipping {table_name}: {str(e)[:80]}")
    
    print(f"\nFound {len(updates)} watermark updates")

In [ ]:
# Voeg toe NA Cell 3, VOOR Cell 4
print("\n🔍 Comparing old vs new watermarks:")
for table, new_wm in updates.items():
    old_wm = tables_map.get(table)
    match = "✓ MATCH" if old_wm == new_wm else "✗ DIFFERENT"
    print(f"  {table}: {old_wm} → {new_wm} {match}")

## Update Configuration

Compares new watermark values with existing ones and updates the configuration dictionary where changes are detected.

In [ ]:
# Update watermarks
changed = 0

for table, new_wm in updates.items():
    old_wm = tables_map.get(table)
    if old_wm != new_wm:
        tables_map[table] = new_wm
        changed += 1
        print(f"  {table}: {old_wm} → {new_wm}")

print(f"✓ Updated {changed} watermarks")

## Save Configuration

Writes the updated configuration back to disk using atomic file replacement to prevent corruption.

In [ ]:
# Write config atomically
if changed > 0:
    tmp_path = wm_configpath_full + ".tmp"
    
    with open(tmp_path, 'w') as f:
        json.dump(cfg, f, indent=2)
    
    os.replace(tmp_path, wm_configpath_full)
    print("✓ Config saved")

## Return Result

Outputs processing summary and exits notebook with result payload for pipeline orchestration.

In [ ]:
# Return result for pipeline
result = {"updates": changed, "source": source, "run_id": run_id}
print(json.dumps(result))

# Exit for notebook orchestration
mssparkutils.notebook.exit(json.dumps(result))